In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open(r'\Users\romo_\.secret\yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Dallas, TX'
TERM = 'BBQ'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Dallas_BBQ.json"
JSON_FILE

'Data/results_in_progress_Dallas_BBQ.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Dallas_BBQ.json already exists. Deleting previous file...
[i] Data/results_in_progress_Dallas_BBQ.json not found. Saving empty list to new file.
- 0 previous results found.


250

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/250 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,sqhaTs9xc3G8OWHihGBipQ,terry-blacks-barbecue-dallas,Terry Black's Barbecue,https://s3-media1.fl.yelpcdn.com/bphoto/LD-QPz...,False,https://www.yelp.com/biz/terry-blacks-barbecue...,1982,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 32.78488, 'longitude': -96.779840...",[delivery],$$,"{'address1': '3025 Main St', 'address2': None,...",+14693990081,(469) 399-0081,6322.907117
1,ycL1qii6EpKXoLgMMboBIQ,pecan-lodge-dallas-3,Pecan Lodge,https://s3-media2.fl.yelpcdn.com/bphoto/8g7yxW...,False,https://www.yelp.com/biz/pecan-lodge-dallas-3?...,6487,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 32.78371943063679, 'longitude': -...",[delivery],$$,"{'address1': '2702 Main St', 'address2': '', '...",+12147488900,(214) 748-8900,6346.090815
2,DtYMJeVbFgGXroWYUX5P4Q,hutchins-bbq-frisco-11,Hutchins BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/Wk_OK8...,False,https://www.yelp.com/biz/hutchins-bbq-frisco-1...,2291,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 33.15463, 'longitude': -96.80443}",[delivery],$$,"{'address1': '9225 Preston Rd', 'address2': ''...",+19723772046,(972) 377-2046,35092.077469
3,CCtb60dWnUn9tXIS26s2Yw,cattleack-barbeque-dallas,Cattleack Barbeque,https://s3-media2.fl.yelpcdn.com/bphoto/zfZat-...,False,https://www.yelp.com/biz/cattleack-barbeque-da...,871,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 32.9339209, 'longitude': -96.8365...",[],$$,"{'address1': '13628 Gamma Rd', 'address2': Non...",+19728050999,(972) 805-0999,11143.408689
4,GC8QmhLYqm5bGNPuyrvFvg,lockhart-smokehouse-dallas,Lockhart Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/7lnUB-...,False,https://www.yelp.com/biz/lockhart-smokehouse-d...,2251,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 32.7493251, 'longitude': -96.828514}",[delivery],$$,"{'address1': '400 W Davis St', 'address2': '',...",+12149445521,(214) 944-5521,10316.560949


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,dMPK1CtkGRVJIV_EcvLhkw,golden-chinese-bbq-arlington,Golden Chinese BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/ixPV-z...,False,https://www.yelp.com/biz/golden-chinese-bbq-ar...,101,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 32.70752031519901, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1818 E Pioneer Pkwy', 'address2'...",+18177953772,(817) 795-3772,30074.354673
996,8xXxC5wnsmdqIIa-6qhdVQ,bols-plano-4,Bols,https://s3-media1.fl.yelpcdn.com/bphoto/H9JrNk...,False,https://www.yelp.com/biz/bols-plano-4?adjust_c...,226,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 33.0277409085941, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '6505 W Park Blvd', 'address2': '...",+12145013333,(214) 501-3333,21322.011095
997,0zbf1wKwGhf7TfLccyop5w,mandarin-by-howard-wang-dallas-2,Mandarin by Howard Wang,https://s3-media1.fl.yelpcdn.com/bphoto/VjKccc...,False,https://www.yelp.com/biz/mandarin-by-howard-wa...,48,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 32.962859, 'longitude': -96.790258}","[delivery, pickup]",NaN,"{'address1': '6859 Arapaho Rd', 'address2': 'S...",+12144846369,(214) 484-6369,13808.060878
998,No38yzwts4LbtysMv1Pq5g,williams-fried-chicken-dallas-17,Williams Fried Chicken,,False,https://www.yelp.com/biz/williams-fried-chicke...,1,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.0,"{'latitude': 32.694978, 'longitude': -96.772653}","[delivery, pickup]",NaN,"{'address1': '4875 Sunnyvale St', 'address2': ...",+12143755055,(214) 375-5055,16252.379247
999,noTVgK8NaWeFixAGlmpVdw,wu-wei-din-chinese-cuisine-plano,Wu Wei Din Chinese Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/A4_LOd...,False,https://www.yelp.com/biz/wu-wei-din-chinese-cu...,1170,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",4.5,"{'latitude': 33.020071, 'longitude': -96.75138...","[delivery, pickup]",$$,"{'address1': '2909 W 15th St', 'address2': Non...",+19729851688,(972) 985-1688,20645.375409


In [9]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

1

In [ ]:
final_df.to_csv("Data/final_df.csv",index=False)